This notebook will walk through analyzing summed subsequence count data 

In [1]:
from pathlib import Path
import scipy.optimize
import matplotlib.pyplot as plt 
import numpy as np
import os
# For data preparation
from PertQuant.simCRN.ml_nupack import gen_complement
from PertQuant.analyzeFASTQ.sum_count_dat import get_count_settings
from PertQuant.analyzeFASTQ.sum_count_dat import get_sum_settings
from PertQuant.analyzeFASTQ.analyze_counts import make_x_fit_arr
# For fitting
from PertQuant.analyzeFASTQ.analyze_counts import power_func
from PertQuant.analyzeFASTQ.analyze_counts import fit_func
from PertQuant.analyzeFASTQ.analyze_counts import calc_sum
from PertQuant.analyzeFASTQ.analyze_counts import sum_fit_func
# For barcode plot labels
from PertQuant.analyzeFASTQ.analyze_counts import autolabel
# For time binned plots
from PertQuant.analyzeFASTQ.analyze_counts import plot_time_binned_fits

In [2]:
# For reading in binned data
from PertQuant.analyzeFASTQ.analyze_counts import read_binned_counts
# For fitting time binned data
from PertQuant.analyzeFASTQ.analyze_counts import time_fit_arr
# For fitting other binned data
from PertQuant.analyzeFASTQ.analyze_counts import bin_fit_arr
# For analyzing fitted binned data
from PertQuant.analyzeFASTQ.analyze_counts import sum_binned_target_and_comp
from PertQuant.analyzeFASTQ.analyze_counts import divide_binned_counts

In [3]:
# For reading in non-binned data
from PertQuant.analyzeFASTQ.analyze_counts import read_counts
# For analyzing fitted non-binned data
from PertQuant.analyzeFASTQ.analyze_counts import sum_target_and_comp
from PertQuant.analyzeFASTQ.analyze_counts import divide_counts

## Data analysis step overview
(1) Reading in the data  
(2) Fitting the data  
(3) Summing the target and complement data  
(4) Whatever else (calculating ratios, plots, etc)

## 1.) Reading in the data

### Get settings

In [4]:
count_folder = os.getcwd() # This assumes the jupyter notebook is placed inside the count folder.
count_folder_list = count_folder.split('/')
count_folder_list.pop()
count_settings_path = '/'.join(count_folder_list)+'/[count_settings.txt]' 
# replace count_settings.txt with the name of your settings file.

In [ ]:
n_targets, target_list, target_lengths, n_barcodes, min_len, handle_repeat_error, repeat_list, n_repeat = \
    get_count_settings(count_settings_path)

### Get binned settings (optional)

In [ ]:
# If the data is binned by time. Change the optional arguments if the data was binned otherwise.
time_step, Pbin, Qbin = get_sum_settings(count_folder, has_time_step=True)
tstep_arr = np.arange(0.0,24*60,time_step) # if run length is 24 hours
num_bins = len(tstep_arr)

### Prepare arrays for fitting

In [ ]:
# Figure out the array length (number of columns)
# Figure out how many subsequence lengths there are
max_num_subseqs = seq_len - min_len + 1
# Calculate the count array length
array_len = int(max_num_subseqs * (max_num_subseqs+1)/2)

In [ ]:
seq_len = len(target_list[0])
x_array, x_fit = make_x_fit_arr(seq_len, min_len)

### Get lists of the count files

In [ ]:
# Get the names of target count files
target_count_files = Path(count_folder).glob("*_[0-9]_counts.txt")
target_file_list = sorted([str(file) for file in target_count_files])
# Get the names of target complement count files
targetc_count_files = Path(count_folder).glob("*_[0-9]_comp_counts.txt")
targetc_file_list = sorted([str(file) for file in targetc_count_files])

### Read in non-binned data

In [ ]:
# Read in the summed subsequence counts
target_file_arr_list = []
targetc_file_arr_list = []
for i in range(len(target_file_list)):
    # Read in the target counts
    target_arr = np.zeros(array_len)
    read_counts(target_file_list[i], target_arr)
    target_file_arr_list.append(target_arr)
    # Read in the target complement counts
    targetc_arr = np.zeros(array_len)
    read_counts(targetc_file_list[i], targetc_arr)
    targetc_file_arr_list.append(targetc_arr)

### Read in binned data (optional)

In [ ]:
# This example uses data binned by time. Replace tstep_arr with the appropriate bin array
# Read in the summed subsequence counts
target_file_arr_list = []
targetc_file_arr_list = []
for i in range(len(target_file_list)):
    # Read in the target counts
    target_arr = np.zeros((len(tstep_arr),array_len))
    read_binned_counts(target_file_list[i], target_arr)
    target_file_arr_list.append(target_arr)
    # Read in the target complement counts
    targetc_arr = np.zeros((len(tstep_arr),array_len))
    read_binned_counts(targetc_file_list[i], targetc_arr)
    targetc_file_arr_list.append(targetc_arr)

## 2.) Fit the data

### Power function fitting

In [ ]:
# Run this function to fit the power function for non-binned data.
fit_params, fit, pcov, perr = fit_func(power_func, x_array, target_counts_array)
# target_counts_array is one of the arrays in target_file_arr_list

### Power function fitting (binned)

In [ ]:
# Run this function to fit the power function for time binned data.
time_summed_counts_arr, params_arr, fit_list, pcov_list, perr_arr = time_fit_arr(target_bin_array, x_array, \
                                                                                 power_func, False, seq_len, \
                                                                                 min_len, array_len)
# target_bin_array is one of the arrays in target_file_arr_list

In [ ]:
# Run this function to fit the power function for other binned data.
time_summed_counts_arr, params_arr, fit_list, pcov_list, perr_arr = bin_fit_arr(target_bin_array, x_array, \
                                                                                power_func, False, seq_len, min_len)
# target_bin_array is one of the arrays in target_file_arr_list

### Define the sum power function
Locally define it, since it requires the global value seq_len

In [5]:
def summed_power_func(x, S, p):
    return S*(seq_len+1-x)*p ** x

### Summed power function fitting

In [ ]:
# Run this function to fit the summed power function for non-binned data.
summed_counts_arr, fit_params, fit, pcov, perr = sum_fit_func(target_counts_array, x_fit, seq_len, min_len, \
                                                              summed_power_func)
# target_counts_array is one of the arrays in target_file_arr_list

### Summed power function fitting (binned)

In [ ]:
# Run this function to fit the summed power function for time binned data.
time_summed_counts_arr, params_arr, fit_list, pcov_list, perr_arr = time_fit_arr(target_bin_array, x_fit, \
                                                                                 summed_power_func, True, seq_len, \
                                                                                 min_len, array_len)
# target_bin_array is one of the arrays in target_file_arr_list

In [ ]:
# Run this function to fit other binned data.
time_summed_counts_arr, params_arr, fit_list, pcov_list, perr_arr = bin_fit_arr(target_bin_array, x_fit, \
                                                                                summed_power_func, True, seq_len, \
                                                                                min_len)
# target_bin_array is one of the arrays in target_file_arr_list

## 3.) Sum the target and complement data

### Sum non-binned data

In [ ]:
tot_S, tot_S_err = sum_target_and_comp(target_params_arr, target_perr_arr, comp_params_arr, comp_perr_arr)

### Sum binned data

In [ ]:
tot_S_arr, tot_S_err_arr = sum_binned_target_and_comp(target_params_arr, target_perr_arr, comp_params_arr, \
                                                      comp_perr_arr)

## 4.) Other things

### Computing ratios

In [ ]:
ratio, ratio_err = divide_counts(dividend_S, dividend_S_err, divisor_S, divisor_S_err)

### Computing binned ratios

In [ ]:
ratio_arr, ratio_err_arr = divide_binned_counts(dividend_S_arr, dividend_S_err_arr, divisor_S_arr, \
                                                divisor_S_err_arr)

## Plots

In [6]:
# Make a save folder for the plots
fit_save_folder = f"{count_folder}/fit plots"
if not os.path.exists(fit_save_folder):
    os.makedirs(fit_save_folder)

### Plotting non-binned power function fits

In [ ]:
plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots()
ax.scatter(x_array, comp_counts, label="counts", marker=".", color="k") # I plot complements in black
# ax.scatter(x_array, target_counts, label="counts", marker=".", color="g") # I plot targets in green
ax.plot(x_array, fit_arr, label ="fit: S = {:.0f} $\pm$ {:.0f}, p = {:.2f} $\pm$ {:.1e}".\
        format(fit_params[0], perr[0], fit_params[1], perr[1]))
ax.legend(bbox_to_anchor=(1.8, 0.6))
fig.suptitle("Power Function Fit of Complement Subsequence Counts")
ax.set_xlabel("Length of Subsequence (max 25)")
ax.set_ylabel("Number of Matches")
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
# ax.set_ylim(top=50000)
ax.set_xticks(np.arange(min(x_fit), max(x_fit)+1, 5.0))
# plt.savefig(count_file_path+"/fastq_pass/yourname_fit.png", bbox_inches='tight')

### Plotting non-binned summed power function fits

In [ ]:
plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots()
ax.scatter(x_array, comp_counts, label="counts", marker=".", color="k") # I plot complements in black
ax.scatter(x_fit, summed_counts_arr, label="counts", marker=".", color="g") # I plot targets in green
ax.plot(x_fit, summed_fit_arr, label ="fit: S = {:.0f} $\pm$ {:.0f}, p = {:.2f} $\pm$ {:.1e}".\
        format(fit_params[0], perr[0], fit_params[1], perr[1]))
ax.legend(bbox_to_anchor=(1.8, 0.6))
fig.suptitle("Power Function Fit of Summed Subsequence Counts")
ax.set_xlabel("Length of Subsequence (max 25)")
ax.set_ylabel("Number of Matches")
# ax.set_ylim(top=650000)
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax.set_xticks(np.arange(min(x_fit), max(x_fit)+1, 5.0))
# plt.savefig(count_file_path+"yourname_sum_fit.png", bbox_inches='tight')

### Plotting non-binned sequence count estimate (S)  bar plots

In [ ]:
sample_labels = ["target", "complement", "total S"]
sample = [target_S, comp_S, total_S]
sample_summed = [target_Ss, comp_Ss, total Ss]
# Error values
sample_errs = [target_S_err, comp_S_err, total_S_err]
sample_summed_errs = [target_Ss_err, comp_Ss_err, total Ss_err]

In [ ]:
plt.rcParams.update({'font.size': 16})
colors = plt.cm.Blues(np.linspace(0,1,4))

x_pos = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(10,7))
rects1 = ax.bar(x_pos - width/2, sample, width, yerr= sample_errs, label='Not Summed', \
               capsize=10.0, color=colors[1])
rects2 = ax.bar(x_pos + width/2, sample_summed, width, yerr=sample_summed_errs,  label='Summed', \
               capsize=10.0, color=colors[2])

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Estimated Strands')
ax.set_title('Estimated Strands of Target and Complement')
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

ax.bar_label(rects1, padding=1)
ax.bar_label(rects2, padding=3)
ax.legend(loc='upper left')

fig.tight_layout()
# plt.savefig(save_folder+"/yourname_counts_barplot.png")

### Plotting time binned power func fits

In [ ]:
plt.rcParams.update({'font.size': 12})
plot_time_binned_fits(target, comp, tstep_arr, x_array, time_summed_counts_arr, params_arr, \
    fit_list, perr_arr, time_step, fit_save_folder, is_sum=False, y_lims=(0,0), is_save=True)
# target is the name of the target, eg "Target 0"
# comp is a bool of whether or not the fit is for a target or a complement.

### Plotting time binned summed power func fits

In [ ]:
plt.rcParams.update({'font.size': 12})
plot_time_binned_fits(target, comp, tstep_arr, x_fit, time_summed_counts_arr, params_arr, \
    fit_list, perr_arr, time_step, fit_save_folder, is_sum=True, y_lims=(0,0), is_save=True)
# target is the name of the target, eg "Target 0"
# comp is a bool of whether or not the fit is for a target or a complement.

### Plotting time binned ratios

In [ ]:
# Make an array of times
plot_time_arr = tstep_arr + 30
# Custom colors
colors = plt.cm.Blues(np.linspace(0,1,10))
# Set font size for plots
plt.rcParams.update({'font.size': 16})

# Plot
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].errorbar(plot_time_arr, ratio_arr, yerr=ratio_err_arr, capsize=3, color=colors[3])
ax[0].plot(plot_time_arr, np.ones(len(plot_time_arr)), color="lightgray", linestyle="dashed" ,label='expected')
# ax[0].set_ylim((0.95,1.05))
xtick = 180
ax[0].set_xticks(np.arange(0,1440+xtick,xtick))
ax[0].set_title("Not Summed")
ax[0].set_ylabel("Estimated Ratio")

ax[1].errorbar(plot_time_arr, ratio_arr_s, yerr=ratio_err_arr_s, capsize=3, color=colors[3])
ax[1].plot(plot_time_arr, np.ones(len(plot_time_arr)), color="lightgray", linestyle="dashed" ,label='expected')
# ax[1].set_ylim((0.95,1.05))
ax[1].set_xticks(np.arange(0,1440+xtick,xtick))
ax[1].set_title("Summed")
plt.legend(loc="best")

fig.suptitle("Estimated Barcode Ratio Over Time",y=0.99)
# add a big axis, hide frame
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.xlabel("Time (minutes)")
# save plot
# plt.savefig(count_folder+"/name_here_estimated_ratio_plot.png", bbox_inches="tight")

### Plotting ratio bar plots

In [ ]:
sample_labels = ["ratio 0", "ratio 0\n(summed)", "ratio 1", "ratio 1\n(summed)"]
sample = [ratio0, ratio0s, ratio1, ratio1s]
sample_errs = [ratio0_err, ratio0s_err, ratio1_err, ratio1s_err]

In [ ]:
x = np.arange(len(sample_labels))  # the label locations
width = 0.35  # the width of the bars
plt.rcParams.update({'font.size': 24})
# Custom colors
colors = plt.cm.Blues(np.linspace(0,1,10))

fig, ax = plt.subplots(figsize=(10,7))
rects1 = ax.bar(x - width/2, sample_0, width, yerr= sample_0_errs, label='Sample 0', \
               capsize=5.0, color=colors[1])
# rects2 = ax.bar(x + width/2, sample_1, width, yerr= sample_1_errs,  label='Sample 1', \
#                capsize=5.0, color=colors[2])

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Barcode Ratios')
ax.set_title('Barcode Ratios by \nCalculation Method')
ax.set_xticks(x)
ax.set_xticklabels(sample_labels)
ax.set_ylim(top=3.5)
xlims = ax.get_xlim()
xdiff=xlims[1]-xlims[0]
ax.set_xlim(xlims)
# Plot lines for expected ratios
ax.plot([xlims[0],xlims[0]+xdiff/2],[2.0,2.0], linestyle="dashed",color="darkgrey", label="Expected")
ax.plot([xlims[0]+xdiff/2,xlims[1]],[0.5,0.5], linestyle="dashed",color="darkgrey",)

ax.bar_label(rects1, padding=1)
ax.bar_label(rects2, padding=3)
ax.legend()

fig.tight_layout()

# plt.savefig(save_folder+"/ratios_barplot.png")